In [2]:
# Question & Answer 평가 예제
#
# => input_ids : [CLS]질문[SEP]지문[SEP]
# => attention_mask : 1111111111(질문, 지문 모두 1)
# => token_type_ids : 0000000(질문)1111111(지문)
# => start_positions : 45 (질문에 대한 지문에서의 답변 시작 위치)
# => end_positions : 60 (질문에 대한 지문에서의 답변 끝 위치)

import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import os
import sys
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering
from tqdm.notebook import tqdm

sys.path.append('..')
from myutils import seed_everything, GPU_info, mlogging, QADataset

logger = mlogging(loggername="bertQAtest", logfilname="bertQAtest")
device = GPU_info()
seed_everything(111)

logfilepath:bwdataset_2022-03-16.log
logfilepath:qnadataset_2022-03-16.log
logfilepath:bertQAtrain_2022-03-16.log
True
device: cuda:0
cuda index: 0
gpu 개수: 1
graphic name: NVIDIA A30


In [3]:
#############################################################################################
# 변수들 설정
# - model_path : from_pretrained() 로 호출하는 경우에는 모델파일이 있는 폴더 경로나 
#          huggingface에 등록된 모델명(예:'bert-base-multilingual-cased')
#          torch.load(model)로 로딩하는 경우에는 모델 파일 풀 경로
#
# - vocab_path : from_pretrained() 호출하는 경우에는 모델파일이 있는 폴더 경로나
#          huggingface에 등록된 모델명(예:'bert-base-multilingual-cased')   
#          BertTokenizer() 로 호출하는 경우에는 vocab.txt 파일 풀 경로,
#
# - OUTPATH : 출력 모델, vocab 저장할 폴더 경로
#############################################################################################

model_path = '../model/distilbert/distilbert-fpt-wiki_20190620-mecab-model-0313-QA-0315'
vocab_path = '../model/distilbert/distilbert-fpt-wiki_20190620-mecab-model-0313-QA-0315'

# tokeniaer 및 model 설정
#tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# strip_accents=False : True로 하면, 가자 => ㄱ ㅏ ㅈ ㅏ 식으로 토큰화 되어 버림(*따라서 한국어에서는 반드시 False)
# do_lower_case=False : # 소문자 입력 사용 안함(한국어에서는 반드시 False)
tokenizer = DistilBertTokenizer.from_pretrained(vocab_path, strip_accents=False, do_lower_case=False) 
   
model = DistilBertForQuestionAnswering.from_pretrained(model_path)

model.to(device)

DistilBertForQuestionAnswering(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(143772, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
           

In [4]:
model.num_parameters()

153340418

In [6]:
# 학습 data loader 생성
sys.path.append('..')
from myutils import KorQuADCorpus, QADataset, data_collator
from torch.utils.data import DataLoader, RandomSampler

#############################################################################
# 변수 설정
#############################################################################
max_seq_len = 128   # 질문 + 지문 최대 크기
max_query_length = 32  # 질문 최대 크기
doc_stride = 64     # 지문이 128을 넘을 경우, 얼만큼씩 다음 지문으로 대체할지

batch_size = 32        # 배치 사이즈(64면 GUP Memory 오류 나므로, 32 이하로 설정할것=>max_seq_length 를 줄이면, 64도 가능함)
cache = True   # 캐쉬파일 생성할거면 True로 
#############################################################################

# corpus 파일 설정
corpus = KorQuADCorpus()

eval_file_fpath = '../korpora/korQuAD/KorQuAD_v1.0_dev.json'
eval_dataset = QADataset(file_fpath=eval_file_fpath, tokenizer=tokenizer, corpus=corpus, max_seq_length=max_seq_len, max_query_length = max_query_length, doc_stride= doc_stride, overwrite_cache=cache)

# 평가 dataloader 생성
eval_loader = DataLoader(eval_dataset, 
                          batch_size=batch_size, 
                          #shuffle=True, # dataset을 섞음
                          sampler=RandomSampler(eval_dataset, replacement=False), #dataset을 랜덤하게 샘플링함
                          collate_fn=data_collator, # dataset을 tensor로 변환(예시 {'input_ids':tensor[0,1,2,3,1,], 'token_type_id:tensor[0,0,0,0,0], 'attention_mask:tensor[1,1,1,1,1], 'labels':tensor[5]}
                          num_workers=4)
print('end eval_loader===========================================================')

print('eval_loader_len: {}'.format(len(eval_loader)))

2022-03-16 11:07:39,203 - qnadataset - INFO - Creating features from dataset file at ../korpora/korQuAD/KorQuAD_v1.0_dev.json
convert squad examples to features: 100%|██| 5533/5533 [00:06<00:00, 806.15it/s]
2022-03-16 11:07:46,625 - qnadataset - INFO - *** Example ***
2022-03-16 11:07:46,627 - qnadataset - INFO - question & context: [CLS] 1989년 6월 30일 평양 ##축 ##전에 대표 ##로 파견 된 인물 ##은 ? [SEP] 1989년 2월 15일 여의도 농민 폭력 시위 ##를 주도 ##한 혐의 ( 폭력 ##행위 ##등 ##처 ##벌 ##에 ##관 ##한 ##법 ##률 ##위 ##반 ) 으로 지명 ##수 ##배 ##되었다 . 1989년 3월 12일 서울 ##지 ##방 ##검 ##찰청 공안 ##부는 임종 ##석 ##의 사전 ##구 ##속 ##영 ##장을 발부 ##받 ##았다 . 같은 해 6월 30일 평양 ##축 ##전에 임수 ##경 ##을 대표 ##로 파견 ##하여 국가 ##보 ##안 ##법 ##위 ##반 혐의 ##가 추가 ##되었다 . 경찰 ##은 12월 18일 ~ 20일 사이 서울 경희대 ##학교 ##에서 임종 ##석 ##이 성명 발표 ##를 추진 ##하고 있다는 첩보 ##를 입수 ##했고 , 12월 18일 오전 7 ##시 40 ##분 경 가스 [SEP]
2022-03-16 11:07:46,628 - qnadataset - INFO - answer: 임수 ##경
2022-03-16 11:07:46,629 - qnadataset - INFO - features: QAFeatures(input_ids=[101, 76485, 17253, 40636, 121340, 70122, 68767, 119

end eval_loader===========================================================
eval_loader_len: 653


In [7]:
# tokenier 테스트
print(len(tokenizer))
print(tokenizer.encode("눈에 보이는 반전이었지만 영화의 흡인력은 사라지지 않았다", "정말 재미있다"))
print(tokenizer.convert_ids_to_tokens(131027))
print(tokenizer.convert_tokens_to_ids('정말'))

143772
[101, 9034, 10530, 119728, 11018, 128441, 10739, 69708, 42428, 10459, 10020, 12030, 28143, 10892, 124227, 12508, 49137, 102, 122108, 131027, 11903, 102]
재미있
122108


In [9]:
# 평가 시작
import time

logger.info(f"=== model: {model_path} ===")
logger.info(f"num_parameters: {model.num_parameters()}")

model.eval()
    
total_test_correct = 0
total_test_len = 0
    
start = time.time()
logger.info(f'---------------------------------------------------------')

for data in tqdm(eval_loader):
        # 입력 값 설정
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_mask'].to(device)
        #token_type_ids = data['token_type_ids'].to(device)  
        start_positions = data['start_positions'].to(device)
        end_positions = data['end_positions'].to(device)
    
        # 손실률 계산하는 부분은 no_grade 시켜서, 계산량을 줄임.
        # => torch.no_grad()는 gradient을 계산하는 autograd engine를 비활성화 하여 
        # 필요한 메모리를 줄이고, 연산속도를 증가시키는 역활을 함
        with torch.no_grad():
            # 모델 실행
            outputs = model(input_ids=input_ids, 
                            attention_mask=attention_mask,
                            #token_type_id=token_type_id,      # distilbert에는 token_type_id가 없음
                            start_positions=start_positions,
                            end_positions=end_positions)
    
            # 출력값 loss,logits를 outputs에서 얻어옴
            loss = outputs.loss
            start_scores = outputs.start_logits
            end_scores = outputs.end_logits
    
            # 총 손실류 구함
            # start 포지션 정확도 구함
            start_pred = torch.argmax(F.softmax(start_scores), dim=1)
            start_correct = start_pred.eq(start_positions)
            
            # end 포지션 정확도 구함
            end_pred = torch.argmax(F.softmax(end_scores), dim=1)
            end_correct = start_pred.eq(end_positions)
            
            # start 포지션과 end 포지션 정확도를 더하고 2로 나줌
            total_test_correct += (start_correct.sum().item() + end_correct.sum().item()) / 2
            total_test_len += len(start_positions)
    
logger.info("Validatation Accuracy:{}".format(total_test_correct / total_test_len))
    
logger.info(f'---------------------------------------------------------')
logger.info(f'=== 처리시간: {time.time() - start:.3f} 초 ===')
logger.info(f'-END-\n')

2022-03-16 11:08:34,356 - bertQAtrain - INFO - === model: ../model/distilbert/distilbert-fpt-wiki_20190620-mecab-model-0313-QA-0315 ===
2022-03-16 11:08:34,358 - bertQAtrain - INFO - num_parameters: 153340418
2022-03-16 11:08:34,360 - bertQAtrain - INFO - ---------------------------------------------------------


  0%|          | 0/653 [00:00<?, ?it/s]

/tmp/ipykernel_43952/2573352785.py:41: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  start_pred = torch.argmax(F.softmax(start_scores), dim=1)
/tmp/ipykernel_43952/2573352785.py:45: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  end_pred = torch.argmax(F.softmax(end_scores), dim=1)
2022-03-16 11:08:46,989 - bertQAtrain - INFO - Validatation Accuracy:0.7665964172813488
2022-03-16 11:08:46,994 - bertQAtrain - INFO - ---------------------------------------------------------
2022-03-16 11:08:46,995 - bertQAtrain - INFO - === 처리시간: 12.635 초 ===
2022-03-16 11:08:46,997 - bertQAtrain - INFO - -END-

